In [27]:
import pandas as pd
df = pd.read_csv("Course Evaluation_Parami.csv")

In [28]:
long_columns = [col for col in df.columns if len(col) >= 12]
short_columns = [col for col in df.columns if len(col) < 12]

new_columns = [
    'Attendance_Percentage',
    'Challenge_Expectation',
    'Effort_Course',
    'Preparedness_Class',
    'Performance_Level',
    'Above_Average_Comparison',
    'Teaching_Clarity',
    'Stimulated_Student_Interest',
    'Effective_Time_Use',
    'Instructor_Availability',
    'Instructor_Caring',
    'Increased_Interest',
    'Confidence_Gained',
    'Importance_of_Learning',
    'Course_Challenge',
    'Skill_Development',
    'Study_Hours_Per_Week',
    'Recommendation',
    'Reason'
]

column_mapping = dict(zip(long_columns, new_columns))
df.rename(columns=column_mapping, inplace=True)

# Mapping categorical to numerical values
mapping = {'strongly agree': 5, 'agree': 4, 'neutral': 3, 'disagree': 2, 'strongly disagree': 1}

# Apply mapping to relevant columns for changing categorical to numerical
columns_to_map = [
    'Challenge_Expectation',
    'Effort_Course',
    'Preparedness_Class',
    'Performance_Level',
    'Above_Average_Comparison',
    'Teaching_Clarity',
    'Stimulated_Student_Interest',
    'Effective_Time_Use',
    'Instructor_Availability',
    'Instructor_Caring',
    'Increased_Interest',
    'Confidence_Gained',
    'Importance_of_Learning',
    'Course_Challenge',
    'Skill_Development'
]

df[columns_to_map] = df[columns_to_map].astype(str).apply(lambda x: x.str.lower())

df[columns_to_map] = df[columns_to_map].applymap(lambda x: mapping.get(x, x))

In [26]:
age_outlier = df['Age'] > 26
study_hours_outliers = df['Study_Hours_Per_Week']>40

df.loc[age_outlier,'Age'] = pd.NA
df.loc[study_hours_outliers,'Study_Hours_Per_Week'] = pd.NA

df[['Age','Study_Hours_Per_Week']]=df[['Age','Study_Hours_Per_Week']].fillna(df[['Age','Study_Hours_Per_Week']].mean())

df['Age'] = df['Age'].astype(int)

#Did the students feel challenged and performed well in the course?
summary_stats = df.describe()
transposed_stats = summary_stats.transpose()
print(transposed_stats)

#Standardise the reason

df['Reason'] = df['Reason'].str.strip()
df['Reason'] = df['Reason'].str.lower()
df['Reason'] = df['Reason'].str.replace('.','', regex = False)

                             count       mean       std   min   25%   50%  \
Age                           11.0  19.090909  1.300350  16.0  18.5  20.0   
Attendance_Percentage         11.0  97.000000  4.266146  88.0  96.0  99.0   
Challenge_Expectation         11.0   4.272727  1.272078   1.0   4.0   5.0   
Effort_Course                 11.0   4.000000  0.774597   3.0   3.5   4.0   
Preparedness_Class            11.0   4.090909  0.831209   3.0   3.5   4.0   
Performance_Level             11.0   2.818182  1.470930   1.0   1.5   3.0   
Above_Average_Comparison      11.0   2.636364  1.361817   1.0   1.5   3.0   
Teaching_Clarity              11.0   4.454545  0.687552   3.0   4.0   5.0   
Stimulated_Student_Interest   11.0   4.363636  0.504525   4.0   4.0   4.0   
Effective_Time_Use            11.0   4.454545  0.687552   3.0   4.0   5.0   
Instructor_Availability       11.0   4.363636  0.674200   3.0   4.0   4.0   
Instructor_Caring             11.0   4.363636  0.674200   3.0   4.0   4.0   

In [29]:
#df.groupby('Reason').mean()
df.groupby(['Reason'])[['Effort_Course','Study_Hours_Per_Week']].mean()
df.groupby(['Reason'])[['Preparedness_Class','Study_Hours_Per_Week']].mean()
df.groupby(['Reason'])[['Study_Hours_Per_Week','Performance_Level']].mean()
df.groupby(['Reason'])[['Preparedness_Class','Performance_Level']].max()

,Preparedness_Class,Performance_Level
Reason,,
Degree requirement,5,4
Interest,5,5
To take opportunity.,4,3


In [ ]:
#Did the students give effort, and consistent preparation for the course?

grouped_means = df.groupby('Reason').mean()

columns_of_interest = ['Effort_Course', 'Preparedness_Class']

print("Mean Values for Effort, Preparedness of the Students:")
print(grouped_means[columns_of_interest])

Mean Values for Effort, Preparedness of the Students:
                     Effort_Course  Preparedness_Class
Reason                                                
degree requirement        3.500000            4.000000
interest                  4.333333            4.333333
to take opportunity       3.666667            3.666667


<ipython-input-7-1e3e22cfda16>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_means = df.groupby('Reason').mean()


In [ ]:
#df.corr()

In [ ]:
#3. Is it true that students study consistently? (correlation between actual study hours and their preparation for class) (Are there any outliers or anomalies in the reported number of hours spent on the course per week? How do these outliers affect the mean value of the study hours?)

grouped_means = df.groupby('Reason').mean()

columns_of_interest = ['Study_Hours_Per_Week', 'Preparedness_Class']

print("Mean Values for Study_Hours_Per_Week, Preparedness_Class the Students:")
print(grouped_means[columns_of_interest])

Mean Values for Study_Hours_Per_Week, Preparedness_Class the Students:
                     Study_Hours_Per_Week  Preparedness_Class
Reason                                                       
degree requirement                   7.05            4.000000
interest                             8.50            4.333333
to take opportunity                  8.00            3.666667


<ipython-input-9-6d36ded74b8d>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_means = df.groupby('Reason').mean()


In [ ]:
#4. Did they perform well as much as they gave effort and consistent preparation? (correlation between study hours, effort for the course, preparation for class, performance)

columns_of_interest = ['Performance_Level','Effort_Course','Preparedness_Class']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Performance_Level','Effort_Course','Preparedness_Class']
correlation = relevant_data.corr()
print(correlation)

                    Performance_Level  Effort_Course  Preparedness_Class
Performance_Level            1.000000       0.351069            0.832766
Effort_Course                0.351069       1.000000            0.465946
Preparedness_Class           0.832766       0.465946            1.000000


In [ ]:
#Checking the students' study hours based on the initial reason to join the course ('To take opportunity')
opportunity_students = df[df['Reason'] == 'to take opportunity']
study_hours_opportunity = opportunity_students['Study_Hours_Per_Week']
print("Study Hours for Students Choosing 'to take opportunity' as the Reason:")
print(study_hours_opportunity)

Study Hours for Students Choosing 'to take opportunity' as the Reason:
0     6.0
2     3.0
5    15.0
Name: Study_Hours_Per_Week, dtype: float64


In [ ]:
#5. Study Hours, Effort, Preparedness and Performance Level of the Students Who Study More Than 8 Hours

high_study_hours_students = df[df['Study_Hours_Per_Week'] > 8]
grouped_means = high_study_hours_students.groupby('Reason').mean()

columns_of_interest = ['Study_Hours_Per_Week','Effort_Course', 'Preparedness_Class','Performance_Level', 'Above_Average_Comparison']

print("Mean Values for Effort, Preparedness, and Performance for Students with Study Hours > 8:")
print(grouped_means[columns_of_interest])


Mean Values for Effort, Preparedness, and Performance for Students with Study Hours > 8:
                     Study_Hours_Per_Week  Effort_Course  Preparedness_Class  \
Reason                                                                         
degree requirement               8.100000       4.000000                 5.0   
interest                        11.333333       4.666667                 4.0   
to take opportunity             15.000000       4.000000                 4.0   

                     Performance_Level  Above_Average_Comparison  
Reason                                                            
degree requirement            4.000000                  4.000000  
interest                      3.333333                  2.666667  
to take opportunity           3.000000                  3.000000  


<ipython-input-12-653a82c8b185>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_means = high_study_hours_students.groupby('Reason').mean()


In [ ]:
#Checking the correlation between Effort, Performance and Preparedness for class
columns_of_interest = ['Effort_Course','Performance_Level','Preparedness_Class','Study_Hours_Per_Week']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Effort_Course','Performance_Level','Preparedness_Class','Study_Hours_Per_Week']
correlation = relevant_data.corr()
print(correlation)

                      Effort_Course  Performance_Level  Preparedness_Class  \
Effort_Course              1.000000           0.351069        4.659456e-01   
Performance_Level          0.351069           1.000000        8.327655e-01   
Preparedness_Class         0.465946           0.832766        1.000000e+00   
Study_Hours_Per_Week       0.418701           0.299498        1.877160e-16   

                      Study_Hours_Per_Week  
Effort_Course                 4.187009e-01  
Performance_Level             2.994979e-01  
Preparedness_Class            1.877160e-16  
Study_Hours_Per_Week          1.000000e+00  


In [ ]:
#Checking the correlation between teacher's effectiveness columns and Increased Interest column
columns_of_interest = ['Teaching_Clarity','Stimulated_Student_Interest','Effective_Time_Use','Instructor_Availability','Instructor_Caring','Increased_Interest']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Teaching_Clarity','Stimulated_Student_Interest','Effective_Time_Use','Instructor_Availability','Instructor_Caring','Increased_Interest']
correlation = relevant_data.corr()
print(correlation)

                             Teaching_Clarity  Stimulated_Student_Interest  \
Teaching_Clarity                     1.000000                     0.628971   
Stimulated_Student_Interest          0.628971                     1.000000   
Effective_Time_Use                   0.365385                     0.340693   
Instructor_Availability              0.470679                     0.454344   
Instructor_Caring                    0.470679                     0.454344   
Increased_Interest                   0.541435                     0.265627   

                             Effective_Time_Use  Instructor_Availability  \
Teaching_Clarity                       0.365385                 0.470679   
Stimulated_Student_Interest            0.340693                 0.454344   
Effective_Time_Use                     1.000000                 0.254951   
Instructor_Availability                0.254951                 1.000000   
Instructor_Caring                      0.470679                 0.780000 

In [ ]:
#Checking the correlation between teacher's effectiveness columns and Confidence Gained column
columns_of_interest = ['Teaching_Clarity','Stimulated_Student_Interest','Effective_Time_Use','Instructor_Availability','Instructor_Caring','Confidence_Gained']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Teaching_Clarity','Stimulated_Student_Interest','Effective_Time_Use','Instructor_Availability','Instructor_Caring','Confidence_Gained']
correlation = relevant_data.corr()
print(correlation)

                             Teaching_Clarity  Stimulated_Student_Interest  \
Teaching_Clarity                     1.000000                     0.628971   
Stimulated_Student_Interest          0.628971                     1.000000   
Effective_Time_Use                   0.365385                     0.340693   
Instructor_Availability              0.470679                     0.454344   
Instructor_Caring                    0.470679                     0.454344   
Confidence_Gained                    0.532939                     0.348006   

                             Effective_Time_Use  Instructor_Availability  \
Teaching_Clarity                       0.365385                 0.470679   
Stimulated_Student_Interest            0.340693                 0.454344   
Effective_Time_Use                     1.000000                 0.254951   
Instructor_Availability                0.254951                 1.000000   
Instructor_Caring                      0.470679                 0.780000 

In [25]:
#Checking the correlation between stimulated student interest & Preparedness class
df.groupby(['Stimulated_Student_Interest'])[['Preparedness_Class']].min()

columns_of_interest = ['Stimulated_Student_Interest','Preparedness_Class']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Stimulated_Student_Interest','Preparedness_Class']
correlation = relevant_data.corr()
print(correlation)

                             Stimulated_Student_Interest  Preparedness_Class
Stimulated_Student_Interest                     1.000000            0.151744
Preparedness_Class                              0.151744            1.000000


In [ ]:
columns_of_interest = ['Effective_Time_Use', 'Increased_Interest']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Teaching_Effectiveness', 'Increased_Interest']
correlation = relevant_data.corr()
print(correlation)
#How did the instructor's effective time use for the class contribute to an increased interest in the data science field among the students?

                        Teaching_Effectiveness  Increased_Interest
Teaching_Effectiveness                1.000000            0.422319
Increased_Interest                    0.422319            1.000000


In [ ]:
#Checking the correlation between teacher's effective time use & student's increased interest for class
columns_of_interest = ['Effective_Time_Use', 'Increased_Interest']
relevant_data = df[columns_of_interest]
relevant_data.columns = ['Effective_Time_Use', 'Increased_Interest']
correlation = relevant_data.corr()
print(correlation)

                    Effective_Time_Use  Increased_Interest
Effective_Time_Use            1.000000            0.422319
Increased_Interest            0.422319            1.000000
